<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CrossValidator로-교차검증-및-파라미터-튜닝-수행" data-toc-modified-id="CrossValidator로-교차검증-및-파라미터-튜닝-수행-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CrossValidator로 교차검증 및 파라미터 튜닝 수행</a></span></li><li><span><a href="#Pipeline-클래스와-결합하기" data-toc-modified-id="Pipeline-클래스와-결합하기-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pipeline 클래스와 결합하기</a></span></li></ul></div>

In [7]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

iris = load_iris()
iris_data = iris.data
iris_label = iris.target

iris_pdf = pd.DataFrame(iris_data, columns=iris.feature_names)
iris_pdf['label'] = iris_label

# convert pandas df to spark df
iris_sdf = spark.createDataFrame(iris_pdf)
iris_sdf.limit(5).show()

+-----------------+----------------+-----------------+----------------+-----+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|label|
+-----------------+----------------+-----------------+----------------+-----+
|              5.1|             3.5|              1.4|             0.2|    0|
|              4.9|             3.0|              1.4|             0.2|    0|
|              4.7|             3.2|              1.3|             0.2|    0|
|              4.6|             3.1|              1.5|             0.2|    0|
|              5.0|             3.6|              1.4|             0.2|    0|
+-----------------+----------------+-----------------+----------------+-----+



## CrossValidator로 교차검증 및 파라미터 튜닝 수행

In [11]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# 1. 데이터 분할
train, test = iris_sdf.randomSplit(weights=[0.8, 0.2], seed=42)

# 2. Feature Vectorization
columns = train.columns[:-1]
vec_assembler = VectorAssembler(inputCols=columns, outputCol='features')

train_vec = vec_assembler.transform(train)
test_vec = vec_assembler.transform(test)

# 3. 모델 정의
dt_clf = DecisionTreeClassifier(featuresCol='features', labelCol='label')

# 4. 튜닝할 하이퍼파라미터 grid 정의
param_grid = ParamGridBuilder().addGrid(dt_clf.maxDepth, [5, 10]) \
                               .addGrid(dt_clf.minInstancesPerNode, [3, 10]) \
                               .build()

# 5. 교차검증 수행하면서 측정할 메트릭 정의
cv_evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

# 6. CV 객체 정의(모델, 파라미터 grid, 메트릭, 폴드 수 설정)
cv = CrossValidator(estimator=dt_clf, estimatorParamMaps=param_grid, evaluator=cv_evaluator, numFolds=3)

# fit 메소드로 하이퍼파라미터 튜닝 및 교차검증 수행 -> 모델 객체 반환(기본적으로 최적의 파라미터로 refit 됨)
cv_model = cv.fit(train_vec)


In [20]:
def get_cv_result_pdf(cv_model):
    params = [{k.name: v for k, v in c.items()} for c in cv_model.getEstimatorParamMaps()]
    result_df = pd.DataFrame({'params': params,
                             'evaluation_result': cv_model.avgMetrics})
    return result_df

In [23]:
# 최적의 하이퍼파라미터로 학습된 cv_model 객체로 테스트 데이터에 대해 예측!
test_pred = cv_model.transform(test_vec)

test_evaluator = MulticlassClassificationEvaluator(labelCol='label',
                                            predictionCol='prediction',
                                            metricName='accuracy')
test_acc = test_evaluator.evaluate(test_pred)
print('Test acc:', test_acc)

Test acc: 1.0


## Pipeline 클래스와 결합하기
- 총 2가지 방법으로 가능
- 파이프라인으로 묶은 후, 이를 CrossValidator에 집어넣기
- CrossValidator까지 과정을 스테이지로 해서 파이프라인으로 집어넣기


In [25]:
# 첫번째 방법
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 1. Train, Test 분할
train, test = iris_sdf.randomSplit(weights=[0.8, 0.2], seed=42)

# 2. Feature Vectorization Stage 생성
columns = train.columns[:-1]
stage_1 = VectorAssembler(inputCols=columns, outputCol='features')

# 3. 모델 stage 생성
stage_2 = DecisionTreeClassifier(featuresCol='features', labelCol='label')

# 4. 2번,3번 파이프라인 생성
pipeline = Pipeline(stages=[stage_1, stage_2])

# 5. 파라미터 grid 생성
param_grid = ParamGridBuilder().addGrid(stage_2.maxDepth, [5, 12]) \
                               .addGrid(stage_2.minInstancesPerNode, [3, 8]) \
                               .build()
# 6. evaluator 생성
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

# 7. CV 객체 생성
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# 8. 학습 및 교차검증, 파라미터 튜닝 수행
cv_model = cv.fit(train)
print('###### 교차검증 및 튜닝 완료 ######')

###### 교차검증 및 튜닝 완료 ######


In [26]:
# 테스트 데이터에 대해 수행
test_pred = cv_model.transform(test)
test_eval = MulticlassClassificationEvaluator(labelCol='label',
                                             predictionCol='prediction',
                                             metricName='accuracy')
test_acc = test_eval.evaluate(test_pred)
print('Test acc:', test_acc)

Test acc: 1.0


---

In [27]:
# 두번째 파이프라인 활용방법 -> 교차검증까지의 단계를 파이프라인으로 넣기
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# 데이터 분할
train, test = iris_sdf.randomSplit([0.7, 0.3], seed=43)

columns = train.columns[:-1]
# 하나의 스테이지 생성
stage_1 = VectorAssembler(inputCols=columns, outputCol='features')

# 하나의 스테이지(모델, 파라미터 튜닝-CV 까지 동시에 수행) 생성
dt_clf = DecisionTreeClassifier(featuresCol='features', labelCol='label')

param_grid = ParamGridBuilder().addGrid(dt_clf.maxDepth, [5, 10]) \
                               .addGrid(dt_clf.minInstancesPerNode, [10, 20]) \
                               .build()
evaluator = MulticlassClassificationEvaluator(labelCol='label',
                                             predictionCol='prediction',
                                             metricName='accuracy')
stage_cv = CrossValidator(estimator=dt_clf, estimatorParamMaps=param_grid,
                         evaluator=evaluator)

# 파이프라인 생성
pipeline = Pipeline(stages=[stage_1, stage_cv])

# fit
pipeline_model = pipeline.fit(train)
print('###### 교차검증 및 튜닝 완료 ######')

###### 교차검증 및 튜닝 완료 ######


In [28]:
# 테스트 데이터에 대해 수행
test_pred = pipeline_model.transform(test)

test_evaluator = MulticlassClassificationEvaluator(labelCol='label',
                                                  predictionCol='prediction',
                                                  metricName='accuracy')
test_acc = test_evaluator.evaluate(test_pred)
print('Test acc:', test_acc)

Test acc: 0.9148936170212766


---

In [29]:
# TrainValidationSplit 이용하기
# -> 단, 학습 데이터 비율을 지정(맨 끝이 무조건 검증 데이터셋으로 떼놓음)
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

vec_assembler = VectorAssembler(inputCols=columns, outputCol='features')
train_vec = vec_assembler.transform(train)

tvs = TrainValidationSplit(estimator=dt_clf, estimatorParamMaps=param_grid,
                          evaluator=evaluator)
tvs_model = tvs.fit(train_vec)
